# <p style="text-align:center;"> **Entrega 4 - Ruta B**</p>
### <p style="text-align:center;"> *Sebastián y Vicente Ramírez*</p>
### <p style="text-align:center;"> **Apertura y limpieza de datasets**</p>
#### Librerías a usar

In [26]:
# !pip install prince # En caso de ser necesario
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.dummy import DummyRegressor
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
import itertools as it
import prince
import warnings

# Para una visualización más limpia (esta parte del código se corre con todo estructurado
# para evitar ignorar los warnings necesarios durante el proceso de desarrollo)
# warnings.filterwarnings("ignore")

# Configuración pandas
pd.set_option('display.max_columns', None)
pd.set_option("max_info_columns", 200)

#### Apertura de los dataset
> Incluye:
> > - grd_2023 (datos principales).
> > - CIE-10 (para diagnósticos).
> > - Hospitales (para decodificarlos).
> > - GRD (Para todos los diagnósticos).

In [27]:
# Lectura de archivos .csv y .xlsx
hospitales = pd.read_csv("./datasets/Hospitales.csv", header=None, sep="|")
cie = pd.read_excel("./datasets/CIE-10.xlsx")
grd = pd.read_parquet("./datasets/grd_2023.parquet")
grd_big = pd.read_parquet("./datasets/GRD.parquet")


display(grd.info())
display(grd_big.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039587 entries, 0 to 1039586
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   COD_HOSPITAL         1039587 non-null  int64         
 1   CIP_ENCRIPTADO       1037952 non-null  float64       
 2   SEXO                 1039548 non-null  object        
 3   FECHA_NACIMIENTO     1039577 non-null  datetime64[ns]
 4   SERVICIO_SALUD       1038846 non-null  object        
 5   TIPO_INGRESO         1039532 non-null  object        
 6   FECHA_INGRESO        1039587 non-null  datetime64[ns]
 7   FECHAALTA            1039587 non-null  datetime64[ns]
 8   TIPOALTA             1039587 non-null  object        
 9   DIAGNOSTICO1         1039587 non-null  object        
 10  IR_29301_COD_GRD     1039557 non-null  float64       
 11  IR_29301_PESO        1039557 non-null  float64       
 12  IR_29301_SEVERIDAD   1039557 non-null  float64       
 1

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039587 entries, 0 to 1039586
Data columns (total 129 columns):
 #    Column                    Non-Null Count    Dtype         
---   ------                    --------------    -----         
 0    COD_HOSPITAL              1039587 non-null  int64         
 1    CIP_ENCRIPTADO            1039587 non-null  object        
 2    SEXO                      1039587 non-null  object        
 3    FECHA_NACIMIENTO          1039577 non-null  datetime64[ns]
 4    ETNIA                     1039587 non-null  object        
 5    PROVINCIA                 1039587 non-null  object        
 6    COMUNA                    1039587 non-null  object        
 7    NACIONALIDAD              1039587 non-null  object        
 8    PREVISION                 1039587 non-null  object        
 9    SERVICIO_SALUD            1039587 non-null  object        
 10   TIPO_PROCEDENCIA          1039587 non-null  object        
 11   TIPO_INGRESO              1039587 n

None

#### Transformaciones de columnas

In [28]:
# Para dejar como flotante CIP_ENCRIPTADO
def transformar(row):
    if row == "DESCONOCIDO":
        return np.nan
    return row

# Para decodificar los 10 diagnósticos
diagnostico = dict(zip(cie["Código"], cie["Descripción"]))

# Para decodificar hospitales
hospital = dict(zip(hospitales[0], hospitales[1]))

# Ajustar tipós de variable por columna para futuro merge
grd_big["CIP_ENCRIPTADO"] = grd_big["CIP_ENCRIPTADO"].apply(transformar)
grd_big["CIP_ENCRIPTADO"] = grd_big["CIP_ENCRIPTADO"].astype(float)
grd["FECHA_NACIMIENTO"] = grd["FECHA_NACIMIENTO"].astype('datetime64[ns]')
grd["FECHA_INGRESO"] = grd["FECHA_INGRESO"].astype('datetime64[ns]')
grd["FECHAALTA"] = grd["FECHAALTA"].astype('datetime64[ns]')

# Columans a eliminar por indice (Ver output anterior)
out = list(grd_big.columns[6:9]) + \
      list(grd_big.columns[10:11]) + \
      list(grd_big.columns[[4, 12, 13, 35]]) + \
      list(grd_big.columns[15:34]) + list(grd_big.columns[37:53]) + list(grd_big.columns[63:])

grd_big.drop(columns = out, axis = 1, inplace = True)

#### Merge de todos los datasets
> En el siguiente orden
> > 1. (grd  $\xleftrightarrow{merge}$  df) $\rightarrow$ grd_main | (Para tener todos los diagnósticos).
> > 2. df.map(diagnosticos) | para tener los diagnósticos en descripción. (hacerlo con las 10 columans)
> > 2. df.map(hospital) | Decodificar hospitales. Dataset completo.

In [29]:
df = grd.merge(grd_big, how = "left")

for i in range(1, 11):
    df[f'DESCRIPCION{i}'] = df[f"DIAGNOSTICO{i}"].map(diagnostico)
    df.drop(columns = f"DIAGNOSTICO{i}", axis = 1, inplace = True)

df["HOSPITAL"] = df["COD_HOSPITAL"].map(hospital)
df.drop(columns = "COD_HOSPITAL", axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1041229 entries, 0 to 1041228
Data columns (total 26 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   CIP_ENCRIPTADO       1039590 non-null  float64       
 1   SEXO                 1041190 non-null  object        
 2   FECHA_NACIMIENTO     1041219 non-null  datetime64[ns]
 3   SERVICIO_SALUD       1040488 non-null  object        
 4   TIPO_INGRESO         1041174 non-null  object        
 5   FECHA_INGRESO        1041229 non-null  datetime64[ns]
 6   FECHAALTA            1041229 non-null  datetime64[ns]
 7   TIPOALTA             1041229 non-null  object        
 8   IR_29301_COD_GRD     1041199 non-null  float64       
 9   IR_29301_PESO        1041199 non-null  float64       
 10  IR_29301_SEVERIDAD   1041199 non-null  float64       
 11  IR_29301_MORTALIDAD  1041199 non-null  float64       
 12  EDAD                 1041219 non-null  float64       
 1

#### Variable "REINGRESO"
> 1. Eliminar los CIP_ENCRIPTADO nulos
> > Si no se sabe que paciente entró, no es posible analizar de manera corercta tal variable. Además, son menos de 1500 valores nulos dentro de más de un millon de datos, por ende, no es una cantidad significativa.
> 2. Ordenar por fecha el dataset.
> > La función "duplicated" revisa el dataset por orden de registro, y el este no está ordenado por fecha de ingreso, por lo que aplicar directamente la función podría generar un reingreso antes de la primera atención.
> 3. Crear columna REINGRESO
> > Aplicando duplicated a la columna CIP_ENCRIPTADO, generando una columna de datos booleanos

In [30]:
# 1.
df.dropna(subset = ["CIP_ENCRIPTADO"], inplace = True)

# 2.
df.sort_values(by= "FECHA_INGRESO", ascending = True, inplace = True)

# 3.
df["REINGRESO"] = df["CIP_ENCRIPTADO"].duplicated()
df.reset_index(drop = True, inplace  = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1039590 entries, 0 to 1039589
Data columns (total 27 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   CIP_ENCRIPTADO       1039590 non-null  float64       
 1   SEXO                 1039551 non-null  object        
 2   FECHA_NACIMIENTO     1039580 non-null  datetime64[ns]
 3   SERVICIO_SALUD       1038851 non-null  object        
 4   TIPO_INGRESO         1039535 non-null  object        
 5   FECHA_INGRESO        1039590 non-null  datetime64[ns]
 6   FECHAALTA            1039590 non-null  datetime64[ns]
 7   TIPOALTA             1039590 non-null  object        
 8   IR_29301_COD_GRD     1039560 non-null  float64       
 9   IR_29301_PESO        1039560 non-null  float64       
 10  IR_29301_SEVERIDAD   1039560 non-null  float64       
 11  IR_29301_MORTALIDAD  1039560 non-null  float64       
 12  EDAD                 1039580 non-null  float64       
 1

#### Tipo de ingreso
filtrado a urgencias

In [31]:
df = df[df["TIPO_INGRESO"] == "URGENCIA"]
df.reset_index(drop = True, inplace = True)

#### Filtrar por región metropolitana
> Para hacer un análisis más acotado. Incluir Provincias de:
> > - Santiago
> > - Talagante
> > - Cordillera
> > - Chacabuco
> > - Maipo
> > - Melipilla

In [32]:
provincias = ["SANTIAGO", "TALAGANTE", "CORDILLERA", "CHACABUCO", "MAIPO", "MELIPILLA"]
df = df[df["PROVINCIA"].isin(provincias)]
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174676 entries, 0 to 174675
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   CIP_ENCRIPTADO       174676 non-null  float64       
 1   SEXO                 174676 non-null  object        
 2   FECHA_NACIMIENTO     174672 non-null  datetime64[ns]
 3   SERVICIO_SALUD       174676 non-null  object        
 4   TIPO_INGRESO         174676 non-null  object        
 5   FECHA_INGRESO        174676 non-null  datetime64[ns]
 6   FECHAALTA            174676 non-null  datetime64[ns]
 7   TIPOALTA             174676 non-null  object        
 8   IR_29301_COD_GRD     174646 non-null  float64       
 9   IR_29301_PESO        174646 non-null  float64       
 10  IR_29301_SEVERIDAD   174646 non-null  float64       
 11  IR_29301_MORTALIDAD  174646 non-null  float64       
 12  EDAD                 174672 non-null  float64       
 13  DIAS_ESTANCIA 

#### Eliminación de las columans faltantes
>Tales como
> > - CIP_ENCRIPTADO
> > - FECHA_NACIMIENTO
> > - TIPO_INGRESO --> solo si filtramos por urgencia
> > - PROVINCIA
> > - FECHA_INGRESO
> > - FECHAALTA
> > - TIPOALTA
> > - IR_29301_COD_GRD
> > - IR_29301_PESO
> > - IR_29301_SEVERIDAD
> > - IR_29301_MORTALIDAD

In [33]:
out = ['CIP_ENCRIPTADO', 'FECHA_NACIMIENTO', 'TIPO_INGRESO', 'PROVINCIA', 'FECHA_INGRESO', 'FECHAALTA', 'TIPOALTA', 'IR_29301_COD_GRD', 'IR_29301_PESO', 'IR_29301_SEVERIDAD', 'IR_29301_MORTALIDAD']

df.drop(columns = out, axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174676 entries, 0 to 174675
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            174676 non-null  object 
 1   SERVICIO_SALUD  174676 non-null  object 
 2   EDAD            174672 non-null  float64
 3   DIAS_ESTANCIA   174676 non-null  int64  
 4   DESCRIPCION1    174663 non-null  object 
 5   DESCRIPCION2    164002 non-null  object 
 6   DESCRIPCION3    147008 non-null  object 
 7   DESCRIPCION4    128294 non-null  object 
 8   DESCRIPCION5    110615 non-null  object 
 9   DESCRIPCION6    94761 non-null   object 
 10  DESCRIPCION7    80627 non-null   object 
 11  DESCRIPCION8    68054 non-null   object 
 12  DESCRIPCION9    56934 non-null   object 
 13  DESCRIPCION10   47375 non-null   object 
 14  HOSPITAL        174676 non-null  object 
 15  REINGRESO       174676 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(13)
memory usag

#### Filtrar DIAS_ESTANCIA según recomendación
> Es decir, eliminar los outliers de más de 15 días de estancia (y los pacientes sin estancia)

In [34]:
df = df[(df["DIAS_ESTANCIA"] > 0) & (df["DIAS_ESTANCIA"] <= 15)]
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138667 entries, 0 to 138666
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            138667 non-null  object 
 1   SERVICIO_SALUD  138667 non-null  object 
 2   EDAD            138665 non-null  float64
 3   DIAS_ESTANCIA   138667 non-null  int64  
 4   DESCRIPCION1    138654 non-null  object 
 5   DESCRIPCION2    129711 non-null  object 
 6   DESCRIPCION3    115177 non-null  object 
 7   DESCRIPCION4    98407 non-null   object 
 8   DESCRIPCION5    82763 non-null   object 
 9   DESCRIPCION6    68993 non-null   object 
 10  DESCRIPCION7    57077 non-null   object 
 11  DESCRIPCION8    46666 non-null   object 
 12  DESCRIPCION9    37754 non-null   object 
 13  DESCRIPCION10   30312 non-null   object 
 14  HOSPITAL        138667 non-null  object 
 15  REINGRESO       138667 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(13)
memory usag

#### Manejo de nulos.
> Se revisarán los nulos de todas las variables excepto de las Descripciopnes 2 hasta la 10, ya que estos nulos son por el hecho de que no se hicieron más diagnósticos, que es válido para el modelo. En otras palabras, variables a revisar:
> > - SEXO
> > - SERVICIO_SALUD
> > - EDAD
> > - DIAS_ESTANCIA
> > - DESCRIPCION1
> > - HOSPITAL
> > - REINGRESO

In [35]:
var = ["SEXO", "SERVICIO_SALUD", "EDAD", "DIAS_ESTANCIA", "DESCRIPCION1", "HOSPITAL", "REINGRESO"]
nulos = 0
for i in var:
    nul = round((df[i].isnull().sum()/len(df))*100, 3)
    nulos += nul
    print(f'Variable: {i}')
    print(f'Número de nulos: {df[i].isnull().sum()}')
    print(f'Porcentaje de nulos: {nul}%')
    print('\n---------------------------------------------------------------------------\n')
print(f"Porcentaje acumulado de nulos: {nulos}%")


Variable: SEXO
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: SERVICIO_SALUD
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: EDAD
Número de nulos: 2
Porcentaje de nulos: 0.001%

---------------------------------------------------------------------------

Variable: DIAS_ESTANCIA
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: DESCRIPCION1
Número de nulos: 13
Porcentaje de nulos: 0.009%

---------------------------------------------------------------------------

Variable: HOSPITAL


Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Variable: REINGRESO
Número de nulos: 0
Porcentaje de nulos: 0.0%

---------------------------------------------------------------------------

Porcentaje acumulado de nulos: 0.009999999999999998%


Como el porcentaje de nulos es de un 0.01% de los datos, se pueden eliminar sin problema.

In [36]:
df.dropna(subset = var, inplace = True)
df.reset_index(drop = True, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138652 entries, 0 to 138651
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            138652 non-null  object 
 1   SERVICIO_SALUD  138652 non-null  object 
 2   EDAD            138652 non-null  float64
 3   DIAS_ESTANCIA   138652 non-null  int64  
 4   DESCRIPCION1    138652 non-null  object 
 5   DESCRIPCION2    129697 non-null  object 
 6   DESCRIPCION3    115164 non-null  object 
 7   DESCRIPCION4    98395 non-null   object 
 8   DESCRIPCION5    82755 non-null   object 
 9   DESCRIPCION6    68986 non-null   object 
 10  DESCRIPCION7    57070 non-null   object 
 11  DESCRIPCION8    46661 non-null   object 
 12  DESCRIPCION9    37750 non-null   object 
 13  DESCRIPCION10   30309 non-null   object 
 14  HOSPITAL        138652 non-null  object 
 15  REINGRESO       138652 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(13)
memory usag

#### Rellenar DESCRIPCION2 a DESCRIPCION10
> Con el valor "N.A." (de "No Aplica")

In [37]:
def noaplica(row):
    if pd.isna(row):
        return "N.A."
    return row
for i in range(2, 11):
    df[f'DESCRIPCION{i}'] = df[f'DESCRIPCION{i}'].apply(noaplica)

df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138652 entries, 0 to 138651
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SEXO            138652 non-null  object 
 1   SERVICIO_SALUD  138652 non-null  object 
 2   EDAD            138652 non-null  float64
 3   DIAS_ESTANCIA   138652 non-null  int64  
 4   DESCRIPCION1    138652 non-null  object 
 5   DESCRIPCION2    138652 non-null  object 
 6   DESCRIPCION3    138652 non-null  object 
 7   DESCRIPCION4    138652 non-null  object 
 8   DESCRIPCION5    138652 non-null  object 
 9   DESCRIPCION6    138652 non-null  object 
 10  DESCRIPCION7    138652 non-null  object 
 11  DESCRIPCION8    138652 non-null  object 
 12  DESCRIPCION9    138652 non-null  object 
 13  DESCRIPCION10   138652 non-null  object 
 14  HOSPITAL        138652 non-null  object 
 15  REINGRESO       138652 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(13)
memory usag

,SEXO,SERVICIO_SALUD,EDAD,DIAS_ESTANCIA,DESCRIPCION1,DESCRIPCION2,DESCRIPCION3,DESCRIPCION4,DESCRIPCION5,DESCRIPCION6,DESCRIPCION7,DESCRIPCION8,DESCRIPCION9,DESCRIPCION10,HOSPITAL,REINGRESO
0,HOMBRE,METROPOLITANO CENTRAL,62.0,15,Infarto subendocárdico agudo del miocardio,Otros glaucomas,Otros delirios,"Trastorno del humor [afectivo], no especificado",Cardiomegalia,Derrame pericárdico (no inflamatorio),Hipertensión esencial (primaria),Diabetes mellitus tipo 2 con complicaciones of...,"Enfermedad renal crónica, estadio 5",Dependencia de diálisis renal,Hospital Clínico Metropolitano El Carmen Doct...,False
1,MUJER,METROPOLITANO OCCIDENTE,56.0,15,Neumonía debida a Klebsiella pneumoniae,Síndrome de dificultad respiratoria del adulto,"Insuficiencia respiratoria aguda, Tipo I (hipo...",Sepsis no especificada,Staphylococcus aureus como causa de enfermedad...,Síndrome de Guillain- Barré,Gastroenteritis y colitis de origen no especif...,Diabetes mellitus tipo 2 con otras complicacio...,Hipertensión esencial (primaria),Otros tipos de obesidad,"Hospital Dr. Félix Bulnes Cerda (Santiago, Qu...",False
2,HOMBRE,METROPOLITANO SUR,82.0,15,"Infección de vías urinarias, sitio no especifi...","Enfermedad inflamatoria de la próstata, no esp...","Infección bacteriana, no especificada",Otros agentes bacterianos especificados como c...,Poliuria,"Hemorragia gastrointestinal, no especificada",Esofagitis,Embolia y trombosis de otras venas especificadas,Hiperplasia de la próstata,Hipertensión esencial (primaria),"Hospital Barros Luco Trudeau (Santiago, San M...",False
3,HOMBRE,METROPOLITANO SUR,41.0,14,"Hemotórax traumático, con herida dentro de la ...","Traumatismo del corazón con hemopericardio, co...",Choque traumático,"Disparo de otras armas de fuego, y las no espe...","Insuficiencia respiratoria aguda, Tipo no espe...",Anemia posthemorrágica aguda,Alcalosis,N.A.,N.A.,N.A.,"Hospital El Pino (Santiago, San Bernardo)",False
4,MUJER,METROPOLITANO CENTRAL,60.0,14,"Fractura pertrocanteriana, cerrada","Caída en el mismo nivel por deslizamiento, tro...",Trastornos mentales y del comportamiento debid...,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,N.A.,Hospital de Urgencia Asistencia Pública Dr. A...,False
